In [ ]:

!pip install optimum


In [ ]:
!pip install onnxruntime


In [ ]:
!pip install onnxruntime-gpu


In [ ]:
!pip install onnx


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from optimum.onnxruntime import ORTTrainer
from optimum.onnxruntime.configuration import ORTConfig
from torch.utils.data import DataLoader
from transformers import Trainer, TrainingArguments
df = pd.read_csv("/content/unique_context_aware_expense_data_50k.csv")

label_map = {label: i for i, label in enumerate(df['Category'].unique())}
df['Category'] = df['Category'].map(label_map)


dataset = Dataset.from_pandas(df)
dataset = dataset.rename_column("Category", "labels")
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["Sentence"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_map))


train_dataloader = DataLoader(tokenized_dataset, batch_size=16)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Fine-tuning with Hugging Face Trainer
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Train the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")
print("Model fine-tuned and saved!")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss
500,0.101800
1000,0.000700
1500,0.000300
2000,0.000200
2500,0.000100
3000,0.000100
3500,0.000100


Step,Training Loss
500,0.101800
1000,0.000700
1500,0.000300
2000,0.000200
2500,0.000100
3000,0.000100
3500,0.000100
4000,0.000100
4500,0.000100
5000,0.000000


Model fine-tuned and saved!


In [ ]:
from transformers import pipeline

# Load the fine-tuned model and tokenizer
classifier = pipeline("text-classification", model="./fine_tuned_model", tokenizer="./fine_tuned_model")



Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
